In [7]:
import numpy as np
import pandas as pd
import os 
import glob

In [85]:
path = "/Users/egordon/Documents/Coding/budget/data/"
os.chdir(path)
files = glob.glob("*.csv") + glob.glob("*.CSV")

In [205]:
order = ['Source','Transaction Date', 'Description', 'Category',  'Amount',
        'Month','Type', 'Exclude', 'Only Exclusions', 'Without Exclusions']
def file_names(file):
    if file.find('Checking') == 0:
        return 'Checking'
    if file.find('Chase') == 0:
        return 'Chase'
    if file.find('Market') == 0:
        return 'Savings'
    if file.find('From') == 0:
        return 'Citi'
    if file.find('CreditCard')==0:
        return 'Wells Credit Card'
    else:
        return 'N/A'    

def checking_exclusions(description):
    """
        Need to Write Rules for Exclusions       
x
    Acorns
    CITI Card
    CHASE CREDIT

N 
    ONLINE TRANSFER 
    RECURRING TRANSFER 
    """
    return np.nan
    
def compile_files( file , file_name ):
    
    if file_name == 'Chase':
        cur_data = pd.read_csv(file)
        cur_data.drop( columns = ['Post Date'],inplace= True )
        cur_data['Exclude'] = cur_data['Type'].replace({'Payment': 'x', 'Sale': ''})
        cur_data['Type'] = cur_data['Type'].replace({'Payment':'Income','Sale':'Spent'})
   
    if file_name == 'Checking':
        cur_data = pd.read_csv(file, header =None)
        cur_data.drop(columns= [2,3], inplace=True)
        cur_data.columns = ['Transaction Date','Amount','Description']
        cur_data['Type'] = cur_data.apply(lambda x: 'Income' if x['Amount'] > 0 else 'Spent', axis =1 )
        
        #Need to Do some Pythong String Searching 
        cur_data['Exclude'], cur_data['Category']  = '', ''
        
    cur_data['Month'] =  pd.DatetimeIndex(cur_data['Transaction Date']).month
    cur_data['Source'] = file_name
    cur_data['Only Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if x['Exclude']== 'x' else 0, axis =1)
    cur_data['Without Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if x['Exclude'] != 'x' else 0, axis =1)     
        
    return cur_data[order]
        

In [206]:
# Chase [3:4]
all_data = []
for file in files[1:2] + files[3:4]:
    all_data.append(compile_files( file, file_names(file)))

pd.concat(all_data)

,Source,Transaction Date,Description,Category,Amount,Month,Type,Exclude,Only Exclusions,Without Exclusions
0,Checking,11/26/2019,VENMO PAYMENT 2783507287 ERIC GORDON,,-60.00,11,Spent,,0.00,-60.00
1,Checking,11/26/2019,CHASE CREDIT CRD EPAY 191125 4412004838 ERIC C...,,-208.00,11,Spent,,0.00,-208.00
2,Checking,11/26/2019,CITI CARD ONLINE PAYMENT 191125 42312933188009...,,-26.77,11,Spent,,0.00,-26.77
3,Checking,11/26/2019,CITI CARD ONLINE PAYMENT 191125 42312933257889...,,-15.38,11,Spent,,0.00,-15.38
4,Checking,11/25/2019,CITI CARD ONLINE PAYMENT 191122 43312677349641...,,-45.33,11,Spent,,0.00,-45.33
5,Checking,11/25/2019,VENMO PAYMENT 2773866675 ERIC GORDON,,-40.00,11,Spent,,0.00,-40.00
6,Checking,11/25/2019,ONLINE TRANSFER REF #IB077BWMN2 TO WELLS FARGO...,,-26.39,11,Spent,,0.00,-26.39
7,Checking,11/25/2019,RECURRING TRANSFER TO GORDON N REF #OP0775KYQY...,,-250.00,11,Spent,,0.00,-250.00
8,Checking,11/25/2019,RECURRING TRANSFER TO GORDON N REF #OP0775KX34...,,-130.00,11,Spent,,0.00,-130.00
9,Checking,11/22/2019,VENMO CASHOUT 2769614478 ERIC GORDON,,26.77,11,Income,,0.00,26.77


In [124]:
?cur_data.replace

In [99]:





valid = {'Chase','Checking'}
    if file_name not in valid:
        raise ValueError("results: status must be one of %r." % valid)

IndentationError: unexpected indent (<ipython-input-99-25bc55409636>, line 2)